[LSP Scale config](#LSP-config)

[IFL IFD config](#IFL-scale)

[Loopback config](#Loopback-interface-config)

[BGP Scale config](#BGP-config)

---
title: "JUNOS scale config"
last_modified_at: 2016-12-16T00:00:02-05:00
categories:
  - Blog
tags:
  - JUNOS
---

In [4]:
from netaddr import *
from jinja2 import Template

In [49]:
class JunosTest:
    def __init__(self):
        self.howmany = int(input('How many iterations:'))
        print(self.howmany)
        
        
    def myloop(self,hname_template):
        
        # variable to decide when to pick next IP range 
        i = 1 
        # Variable to generate given scale config 
        j = 1 
        ip = IPNetwork('192.168.1.0/24')
        while (self.howmany+1) > j:
            for ips in ip.iter_hosts():
                print (hname_template.render(hostname = ips, hostname2 = i, lid = j))
                i += 1
                if i == 254:
                    ip = ip.next()
                    print(i)
                    i = 0
                j += 1
                if j == self.howmany+1:
                    break
                
    def LspConfig(self):
        #cmd = "set protocols mpls label-switched-path LSP-{} to {{ipadd}} ldp-tunneling"
        hname_template = Template("set protocols mpls label-switched-path LSP-{{lid}} to {{hostname}} ldp-tunneling")
        self.myloop(hname_template)
    
    def IflScale(self):
        print("set interfaces xe-4/1/0 flexible-vlan-tagging encapsulation flexible-ethernet-services ")
        hname_template = Template("set interfaces xe-4/1/0 uni {{lid}} vlan-id {{lid}} family inet address {{hostname}}/24")
        self.myloop(hname_template)
        
    def LoopBack(self):
        cmd = "set interfaces lo0 unit {} family inet address 192.168.1.{}/32"
        hname_template = Template("set interfaces lo0 unit {{lid}} family inet address 192.168.1.{{hostname}}/32")
        self.myloop(hname_template)
        
    def BgpConfig(self):
        cmd = "set protocols bgp group ibpg neighbor 192.168.1.{}"
        hname_template = Template("set protocols bgp group ibpg neighbor {{hostname}}")
        self.myloop(hname_template)

In [46]:
config = JunosTest()

How many iterations: 3


3


# LSP config

In [47]:
config.LspConfig()

set protocols mpls label-switched-path LSP-1 to 192.168.1.1 ldp-tunneling
set protocols mpls label-switched-path LSP-2 to 192.168.1.2 ldp-tunneling
set protocols mpls label-switched-path LSP-3 to 192.168.1.3 ldp-tunneling


# IFL scale

In [48]:
config.IflScale()

set interfaces xe-4/1/0 flexible-vlan-tagging encapsulation flexible-ethernet-services 
set interfaces xe-4/1/0 uni 1 vlan-id 1 family inet address 192.168.1.1/24
set interfaces xe-4/1/0 uni 2 vlan-id 2 family inet address 192.168.1.2/24
set interfaces xe-4/1/0 uni 3 vlan-id 3 family inet address 192.168.1.3/24


# Loopback interface config

In [34]:
config.LoopBack()

set interfaces lo0 unit 1 family inet address 192.168.1.192.168.1.1/32
set interfaces lo0 unit 2 family inet address 192.168.1.192.168.1.2/32
set interfaces lo0 unit 3 family inet address 192.168.1.192.168.1.3/32


# BGP config
<a id = "BGP config"></a>

In [35]:
config.BgpConfig()

set protocols bgp group ibpg neighbor 192.168.1.1
set protocols bgp group ibpg neighbor 192.168.1.2
set protocols bgp group ibpg neighbor 192.168.1.3
